# Feature Selection Using AIC

### Overview

Feature Selection using AIC algorithm

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [16]:
import os
import urllib.request

data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)

data_location: house-sales-full.csv


In [17]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
11148,11149,2/6/07,250000,3356406865,Single Family,2/1/07,427800,0.98,"254,324.00",1,...,1.50,3,7,1960,0,0,59000,97000,98001,False
3129,3130,5/25/06,408000,1214700040,Single Family,5/1/06,400600,0.92,"443,239.00",1,...,3.25,6,7,1963,0,0,91000,267000,98148,False
3166,3167,12/1/10,415000,1224049058,Single Family,12/1/10,330600,0.76,"546,304.00",1,...,2.00,3,8,1953,0,3,432000,191000,98040,False
14930,14931,5/5/06,249950,4392200015,Single Family,5/1/06,400600,0.92,"271,538.00",1,...,1.00,3,6,1977,0,0,81000,99000,98010,False
24702,24703,8/6/08,545000,8926100386,Single Family,8/1/08,397900,0.91,"596,089.00",1,...,1.50,2,7,1944,0,0,359000,205000,98115,False
23409,23410,10/18/07,230000,8637100420,Single Family,10/1/07,431300,0.99,"232,080.00",1,...,2.00,3,6,1994,0,0,80000,128000,98055,False
2969,2970,6/24/10,285000,1138000320,Single Family,6/1/10,345800,0.79,"358,681.00",1,...,1.00,3,7,1972,0,0,241000,142000,98034,False
22849,22850,12/7/06,647000,8106301140,Single Family,12/1/06,423400,0.97,"665,032.00",1,...,2.50,4,9,2006,0,0,131000,330000,98055,False
14577,14578,11/3/06,398000,4302700055,Single Family,11/1/06,421200,0.97,"411,229.00",1,...,1.75,4,7,1954,0,0,115000,159000,98106,False
78,79,11/3/06,200000,40000961,Single Family,11/1/06,421200,0.97,"206,648.00",1,...,1.00,1,6,1943,0,0,104000,70000,98168,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [18]:
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [19]:
house_prices.corr()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [20]:
input_columns= ['zhvi_px', 'zhvi_idx', 'NbrLivingUnits', 'SqFtLot', 'SqFtTotLiving', 'SqFtFinBasement', 'Bathrooms', 
                'Bedrooms', 'BldgGrade',   'YrBuilt', 'YrRenovated', 'TrafficNoise', 'LandVal', 'ImpsVal', 'ZipCode', 'NewConstruction']

label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice
0,405100,0.93,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False,280000
1,404400,0.93,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True,1000000
2,425600,0.98,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False,745000
3,418400,0.96,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False,425000
4,351600,0.81,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,332400,0.76,1,1161,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,False,374000
27059,339400,0.78,1,1005,1410,510,1.75,2,9,2011,0,0,147000,269000,-1,True,374000
27060,325300,0.75,1,11170,1070,0,1.00,4,6,1971,0,0,92000,130000,98055,False,165000
27061,400600,0.92,1,6223,1345,0,2.00,3,7,1939,0,0,103000,212000,98166,False,315000


In [21]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 16)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [22]:
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import  LassoLarsIC
import numpy as np


# algo + data --> model
# algo =  LinearRegression ()
algo = LassoLarsIC(criterion='aic', max_iter=500)

model = algo.fit(x, np.ravel(y))  # <--- training
model


/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/linear_model/_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:652: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 5 iterations, i.e. alpha=1.429e+02, with an active se

LassoLarsIC()

## Step 5 : Predict

In [23]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([346355.95, 781642.97, 481255.64, ..., 157495.21, 326819.34,
       555163.09])

In [24]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a.sample(10)

,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,SalePrice,predicted_price
15935,387600,0.89,1,10250,5480,1470,5.00,5,10,2007,0,0,535000,1057000,98004,False,595000,"1,381,143.73"
4425,400600,0.92,1,9750,1510,0,2.50,3,7,1990,0,0,90000,158000,98038,False,279200,"289,644.25"
7485,339400,0.78,1,4275,1630,0,1.75,2,7,1908,0,1,260000,226000,98115,False,418000,"429,113.99"
11021,430900,0.99,1,5400,1580,0,1.75,4,7,1971,0,0,229000,105000,98056,False,391000,"404,494.40"
14556,434200,1.00,1,5495,1610,0,1.75,3,7,1970,0,0,150000,171000,98108,True,354000,"406,254.26"
15624,423400,0.97,1,4907,2610,0,2.50,3,8,2006,0,0,169000,287000,-1,False,489950,"475,895.17"
19558,429600,0.99,1,5205,860,0,1.00,3,5,1942,0,0,138000,38000,98056,False,235000,"228,657.43"
13144,400600,0.92,1,6625,2830,1180,2.75,4,7,1948,0,0,85000,212000,98178,False,395500,"341,114.48"
1389,381600,0.88,1,4764,1820,0,2.50,4,9,2008,0,2,414000,404000,98004,True,721800,"776,180.43"
2980,397900,0.91,1,7965,1230,0,1.75,3,7,1971,0,0,241000,158000,98034,False,340000,"409,205.57"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [25]:
## coefficients
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [    0.95     0.    1395.52     0.35    25.13   -13.78 15768.25 -1073.83
 12353.27  -115.74    -0.29     0.       0.92     0.6      0.23 29852.84]
intercept :  -227573.3766786561


In [26]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_})
coef

,input_column,coefficient
0,zhvi_px,0.95
1,zhvi_idx,0.00
2,NbrLivingUnits,"1,395.52"
3,SqFtLot,0.35
4,SqFtTotLiving,25.13
5,SqFtFinBasement,-13.78
6,Bathrooms,"15,768.25"
7,Bedrooms,"-1,073.83"
8,BldgGrade,"12,353.27"
9,YrBuilt,-115.74


In [27]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.8402893705203509
MSE :  18769514363.540684
RMSE :  137001.87722633834
